In [1]:
import pandas as pd
import numpy as np
import nltk
import nltk.tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [4]:
dt = pd.read_csv("./train.csv")
dt

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [3]:
ndt = dt.drop(['id','lang_abv'],axis=1)
ndt

,premise,hypothesis,language,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,English,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,English,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,French,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,English,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,Thai,1
...,...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,English,2
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,English,0
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",English,2
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,English,2


In [4]:
count_of_sevens = (dt['language'] == "English").sum()
df = ndt[ndt['language'] == 'English']
df

,premise,hypothesis,language,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,English,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,English,2
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,English,0
7,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,English,2
8,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,English,1
...,...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,English,2
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,English,0
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",English,2
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,English,2


In [6]:
X = df[['premise','hypothesis']]
y = df[['label']]




In [7]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление пунктуации
    text = re.sub(r'[^\w\s]', '', text)
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Объединение токенов обратно в текст
    processed_text = ' '.join(tokens)
    return processed_text

# Применение предобработки ко всей колонке



In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
X_test

,premise,hypothesis
8611,There are many homes built into the hillsides;...,The remaining homes that have not been convert...
9396,Each of them was as tough as a thick tree and ...,They were tough and loyal.
3954,i understand i can imagine you all have much t...,"yes, up there the insects must be really annoy..."
2630,"However, the other young lady was most kind.",I was told to leave immediately by the other y...
4433,well what is it,Don't tell me about it.
...,...,...
8120,HE KNOWS ABOUT THE MINES.,He has no idea that the mines exist.
9609,"If I had chosen to be an actor, I should have ...","I did not choose to become an actor, as at the..."
11457,At the top of the hill is the imposing medieva...,The medieval castle of Kadifekale is located a...
11580,"She admits to Dorcas, 'I don't know what to do...",She did not admit anything while speaking to D...


In [10]:
vectorizer = TfidfVectorizer()
label_encoder = LabelEncoder()
X_train['fit'] = (X_train['premise'] + ' ' + X_train['hypothesis']).apply(preprocess_text)
X_test['fit'] = (X_test['premise'] + ' ' + X_test['hypothesis']).apply(preprocess_text)

X_train_tfidf = vectorizer.fit_transform(X_train['fit'])
X_test_tfidf = vectorizer.transform(X_test['fit'])
y_train_encoded = label_encoder.fit_transform(y_train) # one-hot encoder
y_test_encoded = label_encoder.transform(y_test)
y_train_encoded.shape # должен быть (5496, 3)

/home/andrew/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/andrew/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


(5496,)

In [172]:
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)
X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_tfidf.toarray(), dtype=torch.float32)
X_train_tensor.sum()
X_test_tensor.sum()


tensor(4288.6714)

In [173]:
bs = 100
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False)


In [174]:
class TextClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 18)
        self.fc2 = nn.Linear(18, 30)
        self.fc3 = nn.Linear(30, 25)
        self.fc4 = nn.Linear(25, 12)
        self.fc5 = nn.Linear(12, output_size)
        

    def forward(self, x):
        for i in (self.fc1, self.fc2, self.fc3, self.fc4, self.fc5):
            x = i(x)
            x = nn.Softmax()(x) 
        return x

In [176]:
model = TextClassifier(input_size=X_train_tensor.shape[1],
                       output_size=len(label_encoder.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)


In [177]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(loss)

/home/andrew/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(1.0990, grad_fn=<NllLossBackward0>)
tensor(1.1007, grad_fn=<NllLossBackward0>)
tensor(1.1020, grad_fn=<NllLossBackward0>)
tensor(1.1010, grad_fn=<NllLossBackward0>)
tensor(1.1045, grad_fn=<NllLossBackward0>)
tensor(1.1031, grad_fn=<NllLossBackward0>)
tensor(1.1053, grad_fn=<NllLossBackward0>)
tensor(1.1049, grad_fn=<NllLossBackward0>)
tensor(1.1044, grad_fn=<NllLossBackward0>)
tensor(1.1104, grad_fn=<NllLossBackward0>)
tensor(1.1040, grad_fn=<NllLossBackward0>)
tensor(1.0983, grad_fn=<NllLossBackward0>)
tensor(1.1000, grad_fn=<NllLossBackward0>)
tensor(1.0983, grad_fn=<NllLossBackward0>)
tensor(1.0984, grad_fn=<NllLossBackward0>)
tensor(1.1041, grad_fn=<NllLossBackward0>)
tensor(1.0992, grad_fn=<NllLossBackward0>)
tensor(1.1033, grad_fn=<NllLossBackward0>)
tensor(1.1026, grad_fn=<NllLossBackward0>)
tensor(1.1069, grad_fn=<NllLossBackward0>)
tensor(1.1059, grad_fn=<NllLossBackward0>)
tensor(1.0982, grad_fn=<NllLossBackward0>)
tensor(1.1037, grad_fn=<NllLossBackward0>)
tensor(1.10

In [178]:
model.eval()
all_predictions = []
with torch.no_grad():
    for inputs, _ in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.numpy())

all_predictions

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [179]:
y_pred = label_encoder.inverse_transform(all_predictions)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [180]:



accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.3027656477438137
